<a href="https://colab.research.google.com/github/DavinciDreams/micro-julia/blob/main/microjulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MicroJulia — A Minimal GPT in Pure Julia

Faithful port of Karpathy's MicroJulia: the most atomic way to train a GPT.  
Everything built from scratch: autograd, transformer, Adam optimizer.  
No external dependencies beyond Julia stdlib.

**Architecture** (following GPT-2 with simplifications):
- Custom scalar autograd engine (`Value` type)
- Single-layer transformer with multi-head attention
- RMSNorm (not LayerNorm), no biases, ReLU (not GELU)
- KV cache for natural causal masking
- Adam optimizer with linear LR decay
- Temperature-controlled generation
- Best-model checkpointing with validation loss tracking

Based on: https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95

## 0. Login & Setup

This cell runs in **Python** to read your Colab secrets and set up credentials.

1. Add secrets via the key icon (🔑) in the left sidebar:
   - `HF_TOKEN` — your HuggingFace access token
   - `WANDB_KEY` — your Weights & Biases API key
   - `HF_REPO` — your model repo (e.g. `LisaMegaWatts/MicroJulia`)
2. Run cells 0–1 (login + install Julia, ~3-5 min)
3. **Runtime → Change runtime type → Julia 1.10**
4. Continue with the remaining Julia cells

In [ ]:
# ── Login to HF + W&B and fetch training data ──
!pip install -q wandb huggingface_hub

import os, pathlib

# ── Read Colab secrets ──
hf_token = ""
wandb_key = ""
hf_repo = ""
try:
    from google.colab import userdata
    try: hf_token = userdata.get("HF_TOKEN")
    except Exception: pass
    try: wandb_key = userdata.get("WANDB_KEY")
    except Exception: pass
    try: hf_repo = userdata.get("HF_REPO")
    except Exception: pass
except ImportError:
    pass

# ── Write tokens + repo to ~/.netrc (persists across kernel switch) ──
netrc_path = pathlib.Path.home() / ".netrc"
netrc_lines = []
if hf_token:
    netrc_lines.extend(["machine huggingface.co", "login hf", "password " + hf_token, ""])
if wandb_key:
    netrc_lines.extend(["machine api.wandb.ai", "login user", "password " + wandb_key, ""])
if hf_repo:
    netrc_lines.extend(["machine hf.repo", "login default", "password " + hf_repo, ""])
if netrc_lines:
    netrc_path.write_text(chr(10).join(netrc_lines))
    netrc_path.chmod(0o600)
    print(f"Credentials saved to {netrc_path}")
else:
    print("No secrets found. Add HF_TOKEN, WANDB_KEY, HF_REPO via key icon.")

# ── Fetch training data from repo ──
DATA_FILE = "/content/aristotle_rhetoric.txt"
if not os.path.exists(DATA_FILE):
    print("Fetching training data...")
    !git clone --depth 1 --filter=blob:none --sparse https://github.com/DavinciDreams/micro-julia.git /content/_microjulia 2>/dev/null || true
    !cd /content/_microjulia && git sparse-checkout set data 2>/dev/null || true
    src = "/content/_microjulia/data/aristotle_rhetoric.txt"
    if os.path.exists(src):
        import shutil
        shutil.copy(src, DATA_FILE)
        print(f"Training data: {DATA_FILE} ({os.path.getsize(DATA_FILE)/1024:.0f} KB)")
    else:
        print("Warning: could not fetch training data from repo.")
        print("Upload aristotle_rhetoric.txt manually to /content/")
else:
    print(f"Training data present: {DATA_FILE}")

# ── Mount Google Drive for persistent checkpoints ──
try:
    from google.colab import drive
    drive.mount("/content/drive")
    drive_dir = "/content/drive/MyDrive/MicroJulia"
    os.makedirs(drive_dir + "/checkpoints", exist_ok=True)
    # Cache training data to Drive
    if os.path.exists(DATA_FILE):
        os.makedirs(drive_dir + "/data", exist_ok=True)
        import shutil
        shutil.copy(DATA_FILE, drive_dir + "/data/aristotle_rhetoric.txt")
    print(f"Drive mounted: {drive_dir}/")
except Exception as e:
    print(f"Drive mount skipped: {e}")

print("\nDone! Now run the next cell to install Julia (~3-5 min).")
print("Then: Runtime > Change runtime type > Julia 1.10")

## 1. Install Julia Kernel

This cell downloads and installs Julia + IJulia. **Takes ~3-5 minutes** on first run.

**After it finishes:**
1. Go to **Runtime → Change runtime type**
2. You may see both "Julia" and "Julia 1.10" — pick **Julia 1.10**
3. Continue running the cells below

In [ ]:
%%shell
set -e

JULIA_VERSION="1.10.5"
JULIA_MINOR="1.10"

if [ ! -d "/usr/local/julia-${JULIA_VERSION}" ]; then
    echo "Downloading Julia ${JULIA_VERSION}..."
    wget -q https://julialang-s3.julialang.org/bin/linux/x64/${JULIA_MINOR}/julia-${JULIA_VERSION}-linux-x86_64.tar.gz
    tar xzf julia-${JULIA_VERSION}-linux-x86_64.tar.gz -C /usr/local/
    rm julia-${JULIA_VERSION}-linux-x86_64.tar.gz
    ln -sf /usr/local/julia-${JULIA_VERSION}/bin/julia /usr/local/bin/julia
    echo "Julia installed."
else
    echo "Julia already installed."
fi

julia -e '
    using Pkg
    Pkg.add("IJulia")
    Pkg.add("JSON3")
    using IJulia
    installkernel("Julia")
'

echo ""
echo "==========================================================="
echo "  Julia kernel installed!                                   "
echo "  Now: Runtime -> Change runtime type -> pick Julia 1.10       "
echo "  Then run the cells below.                              "
echo "==========================================================="

## 1b. W&B + HuggingFace Helpers (Julia)

W&B logging uses a persistent Python subprocess fed JSON lines from Julia.  
HuggingFace helpers use `huggingface-cli` to push/pull checkpoints.  
Credentials were saved to disk by the login cell above.

In [ ]:
# ── Ensure pip-installed binaries (huggingface-cli, wandb) are on PATH ──
for p in ["/usr/local/bin", joinpath(homedir(), ".local/bin"), "/root/.local/bin"]
    if isdir(p) && !occursin(p, get(ENV, "PATH", ""))
        ENV["PATH"] = p * ":" * get(ENV, "PATH", "")
    end
end

# ── Read credentials + config from ~/.netrc (written by Python login cell) ──
function load_netrc_tokens()
    netrc = expanduser("~/.netrc")
    tokens = Dict{String,String}()
    if !isfile(netrc)
        return tokens
    end
    lines = readlines(netrc)
    current_machine = ""
    for line in lines
        line = strip(line)
        m = match(r"^machine\s+(\S+)", line)
        if m !== nothing
            current_machine = m.captures[1]
        end
        m = match(r"^password\s+(\S+)", line)
        if m !== nothing
            tokens[current_machine] = m.captures[1]
        end
    end
    return tokens
end

netrc_tokens = load_netrc_tokens()

# W&B
wandb_key = get(netrc_tokens, "api.wandb.ai", "")
if !isempty(wandb_key)
    ENV["WANDB_API_KEY"] = wandb_key
    println("W&B API key: found")
else
    println("W&B API key: not found (run Python login cell first)")
end

# HuggingFace token
hf_token = get(netrc_tokens, "huggingface.co", "")
if !isempty(hf_token)
    ENV["HF_TOKEN"] = hf_token
    println("HF token: found")
else
    println("HF token: not found (run Python login cell first)")
end

# HuggingFace repo ID
HF_REPO_ID = get(netrc_tokens, "hf.repo", "")
if !isempty(HF_REPO_ID)
    println("HF repo: ", HF_REPO_ID)
else
    println("HF repo: not set (add HF_REPO to Colab secrets or set HF_REPO_ID manually)")
end

In [ ]:
# ═══════════════════════════════════════════════════════════════
# W&B logging via persistent Python subprocess
# ═══════════════════════════════════════════════════════════════

WANDB_PROJECT = "microgpt-philosophy"
WANDB_RUN_ID = "microgpt-" * join(rand('a':'z', 6))

# Write a tiny Python helper that reads JSON lines on stdin
write("_wandb_log.py", """
import wandb, json, sys, os
project = os.environ.get("WANDB_PROJECT", "microgpt-philosophy")
run_id = os.environ.get("WANDB_RUN_ID", None)
run = wandb.init(project=project, id=run_id, resume="allow",
                 config={"model": "microgpt", "architecture": "1-layer transformer"})
print(f"W&B run: {run.url}", flush=True)
for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    try:
        data = json.loads(line)
        wandb.log(data)
    except Exception as e:
        print(f"wandb log error: {e}", file=sys.stderr, flush=True)
wandb.finish()
""")

wandb_proc = nothing

function wandb_init()
    global wandb_proc, WANDB_PROJECT, WANDB_RUN_ID
    if !haskey(ENV, "WANDB_API_KEY") || isempty(ENV["WANDB_API_KEY"])
        println("W&B: skipped (no API key)")
        return
    end
    ENV["WANDB_PROJECT"] = WANDB_PROJECT
    ENV["WANDB_RUN_ID"] = WANDB_RUN_ID
    wandb_proc = open(`python3 _wandb_log.py`, "r+")
    println("W&B: initialized ($WANDB_PROJECT / $WANDB_RUN_ID)")
end

function wandb_log(; kwargs...)
    global wandb_proc
    wandb_proc === nothing && return
    metrics = Dict(string(k) => v for (k, v) in kwargs)
    try
        println(wandb_proc, JSON3.write(metrics))
        flush(wandb_proc)
    catch e
        println("W&B log error: $e")
    end
end

function wandb_finish()
    global wandb_proc
    wandb_proc === nothing && return
    try close(wandb_proc) catch end
    wandb_proc = nothing
    println("W&B: run finished")
end

# ═══════════════════════════════════════════════════════════════
# HuggingFace Hub helpers
# ═══════════════════════════════════════════════════════════════

function hf_push(repo_id::String, local_path::String; remote_path::String="")
    rp = isempty(remote_path) ? basename(local_path) : remote_path
    run(`huggingface-cli upload $repo_id $local_path $rp`)
    println("Pushed $local_path -> $repo_id/$rp")
end

function hf_pull(repo_id::String, remote_path::String; local_dir::String="checkpoints")
    mkpath(local_dir)
    run(`huggingface-cli download $repo_id $remote_path --local-dir $local_dir`)
    println("Pulled $repo_id/$remote_path -> $local_dir/")
end

function hf_push_checkpoint(repo_id::String; checkpoint_path::String="checkpoints/best_model.json")
    isfile(checkpoint_path) || error("Checkpoint not found: $checkpoint_path")
    hf_push(repo_id, checkpoint_path)
end

function hf_create_repo(repo_id::String)
    try
        run(`huggingface-cli repo create $repo_id --type model`)
        println("Created HF repo: $repo_id")
    catch
        println("HF repo already exists or creation skipped: $repo_id")
    end
end

println("W&B + HuggingFace helpers defined")

---
## 1. Autograd Engine

Scalar `Value` type with automatic differentiation.  
Each operation creates a node in the computation graph with local gradients stored for backpropagation.

In [ ]:
using Pkg
Pkg.add("JSON3")

using Random
using Printf
using JSON3

Random.seed!(42)

In [ ]:
mutable struct Value
    data::Float64
    grad::Float64
    _children::Vector{Value}
    _local_grads::Vector{Float64}
end

Value(x::Real) = Value(Float64(x), 0.0, Value[], Float64[])

# --- Core operations ---

function Base.:+(a::Value, b::Value)
    Value(a.data + b.data, 0.0, [a, b], [1.0, 1.0])
end

function Base.:*(a::Value, b::Value)
    Value(a.data * b.data, 0.0, [a, b], [b.data, a.data])
end

function Base.:-(a::Value, b::Value)
    Value(a.data - b.data, 0.0, [a, b], [1.0, -1.0])
end

function Base.:/(a::Value, b::Value)
    Value(a.data / b.data, 0.0, [a, b], [1.0 / b.data, -a.data / b.data^2])
end

function Base.:^(a::Value, n::Real)
    Value(a.data^n, 0.0, [a], [n * a.data^(n - 1)])
end

function Base.log(a::Value)
    Value(log(a.data), 0.0, [a], [1.0 / a.data])
end

function Base.exp(a::Value)
    e = exp(a.data)
    Value(e, 0.0, [a], [e])
end

function relu(a::Value)
    Value(max(0.0, a.data), 0.0, [a], [Float64(a.data > 0)])
end

# --- Scalar promotion ---

Base.:+(a::Value, b::Real) = Value(a.data + b, 0.0, [a], [1.0])
Base.:+(a::Real, b::Value) = Value(a + b.data, 0.0, [b], [1.0])
Base.:*(a::Value, b::Real) = Value(a.data * b, 0.0, [a], [Float64(b)])
Base.:*(a::Real, b::Value) = Value(a * b.data, 0.0, [b], [Float64(a)])
Base.:-(a::Value, b::Real) = Value(a.data - b, 0.0, [a], [1.0])
Base.:-(a::Real, b::Value) = Value(a - b.data, 0.0, [b], [-1.0])
Base.:/(a::Value, b::Real) = Value(a.data / b, 0.0, [a], [1.0 / b])
Base.:/(a::Real, b::Value) = Value(a / b.data, 0.0, [b], [-a / b.data^2])
Base.:-(a::Value) = Value(-a.data, 0.0, [a], [-1.0])

Base.zero(::Type{Value}) = Value(0.0)
Base.isless(a::Value, b::Value) = a.data < b.data

println("Value autograd type defined")

In [ ]:
function backward!(loss::Value)
    topo = Value[]
    visited = Set{UInt64}()
    function build_topo(v)
        id = objectid(v)
        id in visited && return
        push!(visited, id)
        for child in v._children
            build_topo(child)
        end
        push!(topo, v)
    end
    build_topo(loss)
    loss.grad = 1.0
    for v in reverse(topo)
        for (child, lg) in zip(v._children, v._local_grads)
            child.grad += lg * v.grad
        end
    end
end

println("backward! defined")

---
## 2. Dataset — Load Training Data

Training data (Aristotle's Rhetoric, 5,478 chunks) is fetched from the GitHub repo during setup.
Falls back to built-in philosophy quotes if no file is found.

In [ ]:
# ── Load training data ──

TRAINING_DATA = String[]

SEARCH_PATHS = [
    "/content/aristotle_rhetoric.txt",
    "/content/_microjulia/data/aristotle_rhetoric.txt",
    "data/aristotle_rhetoric.txt",
    "aristotle_rhetoric.txt",
    "train.txt",
]

for candidate in SEARCH_PATHS
    if isfile(candidate)
        TRAINING_DATA = filter(!isempty, strip.(readlines(candidate)))
        println("Loaded $(length(TRAINING_DATA)) docs from $candidate")
        break
    end
end

# Also check Google Drive for cached data
if isempty(TRAINING_DATA)
    drive_path = "/content/drive/MyDrive/MicroJulia/data/aristotle_rhetoric.txt"
    if isfile(drive_path)
        TRAINING_DATA = filter(!isempty, strip.(readlines(drive_path)))
        println("Loaded $(length(TRAINING_DATA)) docs from Google Drive")
    end
end

# Fall back to built-in quotes
if isempty(TRAINING_DATA)
    TRAINING_DATA = [
        "waste no more time arguing about what a good man should be",
        "the happiness of your life depends upon the quality of your thoughts",
        "you have power over your mind not outside events",
        "very little is needed to make a happy life",
        "the soul becomes dyed with the color of its thoughts",
        "when you arise in the morning think of what a privilege it is to be alive",
        "the best revenge is to be unlike him who performed the injury",
        "accept the things to which fate binds you",
        "if it is not right do not do it if it is not true do not say it",
        "look well into thyself there is a source of strength",
        "do every act of your life as though it were the very last act of your life",
        "it is not death that a man should fear but never beginning to live",
        "we suffer more often in imagination than in reality",
        "true happiness is to enjoy the present without anxious dependence upon the future",
        "it is not because things are difficult that we do not dare",
        "it is because we do not dare that they are difficult",
        "luck is what happens when preparation meets opportunity",
        "begin at once to live and count each separate day as a separate life",
        "the whole future lies in uncertainty live immediately",
        "sometimes even to live is an act of courage",
        "the unexamined life is not worth living",
        "i know that i know nothing",
        "wonder is the beginning of wisdom",
        "knowing yourself is the beginning of all wisdom",
        "happiness depends upon ourselves",
        "it is the mark of an educated mind to entertain a thought without accepting it",
        "we are what we repeatedly do excellence then is not an act but a habit",
        "the whole is greater than the sum of its parts",
        "nature does nothing in vain",
        "the journey of a thousand miles begins with a single step",
        "the mind is not a vessel to be filled but a fire to be kindled",
    ]
    println("No training file found - using $(length(TRAINING_DATA)) built-in philosophy quotes")
    println("Searched paths:")
    for p in SEARCH_PATHS
        status = isfile(p) ? "found" : "not found"
        println("  $p -> $status")
    end
end

---
## 3. Neural Network Primitives + Helpers

All operate on vectors of `Value` — linear layer, softmax, RMSNorm, plus helper functions for checkpoint support.

In [ ]:
function linear(x::Vector{Value}, w::Vector{Vector{Value}})
    [sum(wi * xi for (wi, xi) in zip(wo, x)) for wo in w]
end

function softmax_v(logits::Vector{Value})
    max_val = maximum(v.data for v in logits)
    exps = [exp(v - max_val) for v in logits]
    total = sum(exps)
    [e / total for e in exps]
end

function rmsnorm(x::Vector{Value})
    ms = sum(xi * xi for xi in x) / length(x)
    scale = (ms + 1e-5) ^ (-0.5)
    [xi * scale for xi in x]
end

# Helper: deterministic parameter key ordering
function get_param_keys(n_layer::Int)
    keys = ["wte", "wpe", "lm_head"]
    for i in 0:n_layer-1
        append!(keys, [
            "layer$i.attn_wq", "layer$i.attn_wk", "layer$i.attn_wv", "layer$i.attn_wo",
            "layer$i.mlp_fc1", "layer$i.mlp_fc2"
        ])
    end
    return keys
end

# Helper: initialize weight matrices
function init_matrix(nout::Int, nin::Int; std=0.08)
    [[Value(randn() * std) for _ in 1:nin] for _ in 1:nout]
end

# Helper: flatten state_dict into params vector
function flatten_params(state_dict, param_keys)
    params = Value[]
    for key in param_keys
        for row in state_dict[key]
            append!(params, row)
        end
    end
    return params
end

println("Neural network primitives + helpers defined")

---
## 4. GPT Forward Pass

Processes one token at a time with KV cache for causal masking.

In [ ]:
function gpt(token_id::Int, pos_id::Int,
             keys::Vector{Vector{Vector{Value}}},
             values::Vector{Vector{Vector{Value}}},
             state_dict::Dict{String, Vector{Vector{Value}}},
             n_layer::Int, n_head::Int, head_dim::Int)

    tok_emb = state_dict["wte"][token_id]
    pos_emb = state_dict["wpe"][pos_id]
    x = [t + p for (t, p) in zip(tok_emb, pos_emb)]
    x = rmsnorm(x)

    for li in 0:n_layer-1
        x_res = x
        x = rmsnorm(x)
        q = linear(x, state_dict["layer$(li).attn_wq"])
        k = linear(x, state_dict["layer$(li).attn_wk"])
        v = linear(x, state_dict["layer$(li).attn_wv"])
        push!(keys[li+1], k)
        push!(values[li+1], v)

        x_attn = Value[]
        for h in 0:n_head-1
            hs = h * head_dim + 1
            he = hs + head_dim - 1
            q_h = q[hs:he]
            k_h = [ki[hs:he] for ki in keys[li+1]]
            v_h = [vi[hs:he] for vi in values[li+1]]
            attn_logits = [sum(q_h[j] * k_h[t][j] for j in 1:head_dim) / sqrt(Float64(head_dim))
                           for t in 1:length(k_h)]
            attn_weights = softmax_v(attn_logits)
            head_out = [sum(attn_weights[t] * v_h[t][j] for t in 1:length(v_h))
                        for j in 1:head_dim]
            append!(x_attn, head_out)
        end
        x = linear(x_attn, state_dict["layer$(li).attn_wo"])
        x = [a + b for (a, b) in zip(x, x_res)]

        x_res = x
        x = rmsnorm(x)
        x = linear(x, state_dict["layer$(li).mlp_fc1"])
        x = [relu(xi) for xi in x]
        x = linear(x, state_dict["layer$(li).mlp_fc2"])
        x = [a + b for (a, b) in zip(x, x_res)]
    end

    logits = linear(x, state_dict["lm_head"])
    return logits
end

println("GPT forward pass defined")

---
## 5. Checkpoint Save/Load

Save and load model weights + optimizer state as JSON.  
Checkpoints saved to Google Drive persist across Colab sessions.

In [ ]:
function save_checkpoint(path::String, state_dict, param_keys, uchars, hyperparams;
                         m_buf=nothing, v_buf=nothing, step::Int=0,
                         lr::Float64=0.01, b1::Float64=0.85, b2::Float64=0.99,
                         best_val_loss::Float64=Inf,
                         train_losses::Vector{Float64}=Float64[],
                         val_losses::Vector{Float64}=Float64[],
                         total_steps::Int=0, num_steps_target::Int=0)

    sd_data = Dict{String,Any}()
    for k in param_keys
        sd_data[k] = [[v.data for v in row] for row in state_dict[k]]
    end

    checkpoint = Dict{String,Any}(
        "uchars" => [string(c) for c in uchars],
        "hyperparams" => hyperparams,
        "state_dict" => sd_data,
        "optimizer" => Dict{String,Any}(
            "m_buf" => m_buf === nothing ? Float64[] : collect(m_buf),
            "v_buf" => v_buf === nothing ? Float64[] : collect(v_buf),
            "step" => step,
            "lr" => lr,
            "beta1" => b1,
            "beta2" => b2
        ),
        "training" => Dict{String,Any}(
            "best_val_loss" => best_val_loss,
            "train_losses" => train_losses,
            "val_losses" => val_losses,
            "total_steps_completed" => total_steps,
            "num_steps_target" => num_steps_target
        )
    )

    mkpath(dirname(path))
    open(path, "w") do f
        JSON3.write(f, checkpoint)
    end
    vl_str = best_val_loss == Inf ? "Inf" : @sprintf("%.4f", best_val_loss)
    println("Checkpoint saved: $path (step $step, best_val_loss=$vl_str)")
end

function load_checkpoint(path::String)
    println("Loading checkpoint from $path ...")
    raw = JSON3.read(read(path, String))

    uchars = [only(String(s)) for s in raw["uchars"]]
    BOS = length(uchars) + 1
    vocab_size = BOS

    hp = raw["hyperparams"]
    n_layer = Int(hp["n_layer"])
    n_embd = Int(hp["n_embd"])
    block_size = Int(hp["block_size"])
    n_head = Int(hp["n_head"])
    head_dim = n_embd ÷ n_head

    state_dict = Dict{String, Vector{Vector{Value}}}()
    for (key, matrix) in pairs(raw["state_dict"])
        state_dict[string(key)] = [[Value(Float64(v)) for v in row] for row in matrix]
    end

    opt = raw["optimizer"]
    m_buf = Float64.(collect(opt["m_buf"]))
    v_buf = Float64.(collect(opt["v_buf"]))
    step = Int(opt["step"])
    lr = Float64(opt["lr"])
    b1 = Float64(opt["beta1"])
    b2 = Float64(opt["beta2"])

    trn = raw["training"]
    best_val_loss = Float64(trn["best_val_loss"])
    train_losses = Float64.(collect(trn["train_losses"]))
    val_losses = Float64.(collect(trn["val_losses"]))
    total_steps = Int(trn["total_steps_completed"])
    num_steps_target = Int(trn["num_steps_target"])

    println("  vocab=$vocab_size, embd=$n_embd, layers=$n_layer, step=$step")

    return (;
        state_dict, uchars, BOS, vocab_size,
        n_layer, n_embd, block_size, n_head, head_dim,
        m_buf, v_buf, step, lr, b1, b2,
        best_val_loss, train_losses, val_losses,
        total_steps, num_steps_target
    )
end

println("Checkpoint save/load defined")

---
## 6. Setup — Dataset, Tokenizer, Parameters

Character-level tokenizer with a BOS token. 90/10 train/val split.

In [ ]:
# ── Dataset with train/val split ──
docs = shuffle(TRAINING_DATA)
split_idx = max(1, Int(floor(0.9 * length(docs))))
train_docs = docs[1:split_idx]
val_docs = docs[split_idx+1:end]
if isempty(val_docs)
    val_docs = docs[max(1, end-4):end]
    train_docs = docs[1:max(1, end-5)]
end
println("train: $(length(train_docs)) docs, val: $(length(val_docs)) docs")

# ── Tokenizer ──
uchars = sort(unique(join(docs)))
BOS = length(uchars) + 1
vocab_size = BOS
println("vocab size: $vocab_size ($(length(uchars)) chars + BOS)")

# ── Hyperparameters ──
n_layer    = 1
n_embd     = 16
block_size = 256
n_head     = 4
head_dim   = n_embd ÷ n_head

hyperparams = Dict{String,Any}(
    "n_layer" => n_layer, "n_embd" => n_embd,
    "block_size" => block_size, "n_head" => n_head
)

# ── Initialize parameters ──
state_dict = Dict{String, Vector{Vector{Value}}}()
state_dict["wte"]     = init_matrix(vocab_size, n_embd)
state_dict["wpe"]     = init_matrix(block_size, n_embd)
state_dict["lm_head"] = init_matrix(vocab_size, n_embd)
for i in 0:n_layer-1
    state_dict["layer$i.attn_wq"]  = init_matrix(n_embd, n_embd)
    state_dict["layer$i.attn_wk"]  = init_matrix(n_embd, n_embd)
    state_dict["layer$i.attn_wv"]  = init_matrix(n_embd, n_embd)
    state_dict["layer$i.attn_wo"]  = init_matrix(n_embd, n_embd)
    state_dict["layer$i.mlp_fc1"]  = init_matrix(4 * n_embd, n_embd)
    state_dict["layer$i.mlp_fc2"]  = init_matrix(n_embd, 4 * n_embd)
end

param_keys = get_param_keys(n_layer)
params = flatten_params(state_dict, param_keys)
println("num params: $(length(params))")

---
## 7. Training Loop with Validation + Best-Model Checkpointing

Adam optimizer with linear LR decay.  
Validates every 50 steps, saves `best_model.json` when val loss improves.  
Periodic checkpoints every 200 steps.

In [ ]:
function compute_val_loss(val_docs, uchars, BOS, block_size, state_dict, n_layer, n_head, head_dim)
    total_loss = 0.0
    total_tokens = 0
    for doc in val_docs
        tokens = vcat([BOS], [findfirst(==(ch), uchars) for ch in doc], [BOS])
        n = min(block_size, length(tokens) - 1)
        kv_keys = [Vector{Vector{Value}}() for _ in 1:n_layer]
        kv_vals = [Vector{Vector{Value}}() for _ in 1:n_layer]
        for pos in 1:n
            token_id = tokens[pos]
            target_id = tokens[pos + 1]
            logits = gpt(token_id, pos, kv_keys, kv_vals, state_dict, n_layer, n_head, head_dim)
            probs = softmax_v(logits)
            total_loss += -log(probs[target_id]).data
            total_tokens += 1
        end
    end
    return total_loss / max(total_tokens, 1)
end

println("compute_val_loss defined")

In [ ]:
# ── Adam optimizer buffers ──
lr, b1, b2, eps = 0.01, 0.85, 0.99, 1e-8
m_buf = zeros(length(params))
v_buf = zeros(length(params))

best_val_loss = Inf

# ── Checkpoint paths (local + Drive for persistence) ──
LOCAL_CKPT = "checkpoints"
DRIVE_CKPT = "/content/drive/MyDrive/MicroJulia/checkpoints"
mkpath(LOCAL_CKPT)

function save_to_drive(local_path::String)
    if isdir(DRIVE_CKPT)
        cp(local_path, joinpath(DRIVE_CKPT, basename(local_path)), force=true)
    end
end


train_loss_history = Float64[]
val_loss_history = Float64[]

# ── Initialize W&B logging (if API key is set) ──
if haskey(ENV, "WANDB_API_KEY") && !isempty(ENV["WANDB_API_KEY"])
    wandb_init()
end

# ── Training loop ──
# Dynamic steps: ~3 epochs over training data, min 1000, max 50000
NUM_EPOCHS = 3
num_steps = clamp(NUM_EPOCHS * length(train_docs), 1000, 50000)
println("--- training $num_steps steps (~$(NUM_EPOCHS) epochs over $(length(train_docs)) docs) ---")
t_start = time()

for step in 1:num_steps
    doc = train_docs[mod1(step, length(train_docs))]
    tokens = vcat([BOS], [findfirst(==(ch), uchars) for ch in doc], [BOS])
    n = min(block_size, length(tokens) - 1)

    # Forward pass
    kv_keys = [Vector{Vector{Value}}() for _ in 1:n_layer]
    kv_vals = [Vector{Vector{Value}}() for _ in 1:n_layer]
    losses = Value[]
    for pos in 1:n
        token_id  = tokens[pos]
        target_id = tokens[pos + 1]
        logits = gpt(token_id, pos, kv_keys, kv_vals, state_dict, n_layer, n_head, head_dim)
        probs  = softmax_v(logits)
        push!(losses, -log(probs[target_id]))
    end
    loss = (1 / n) * sum(losses)
    push!(train_loss_history, loss.data)

    # Backward pass
    backward!(loss)

    # Adam update with linear LR decay
    lr_t = lr * (1 - (step - 1) / num_steps)
    for (i, p) in enumerate(params)
        m_buf[i] = b1 * m_buf[i] + (1 - b1) * p.grad
        v_buf[i] = b2 * v_buf[i] + (1 - b2) * p.grad^2
        m_hat = m_buf[i] / (1 - b1^step)
        v_hat = v_buf[i] / (1 - b2^step)
        p.data -= lr_t * m_hat / (sqrt(v_hat) + eps)
        p.grad = 0.0
    end

    # Validate + checkpoint every 50 steps
    if step % 50 == 0
        val_loss = compute_val_loss(val_docs, uchars, BOS, block_size, state_dict, n_layer, n_head, head_dim)
        push!(val_loss_history, val_loss)
        elapsed = time() - t_start

        # Log to W&B
        wandb_log(; step=step, train_loss=loss.data, val_loss=val_loss, lr=lr_t)

        improved = ""
        if val_loss < best_val_loss
            best_val_loss = val_loss
            save_checkpoint("checkpoints/best_model.json", state_dict, param_keys, uchars, hyperparams;
                m_buf=m_buf, v_buf=v_buf, step=step,
                lr=lr, b1=b1, b2=b2,
                best_val_loss=best_val_loss,
                train_losses=train_loss_history, val_losses=val_loss_history,
                total_steps=step, num_steps_target=num_steps)
            improved = " << new best!"
        end

        @printf("step %4d / %4d | train %.4f | val %.4f | %.1fs%s\n",
                step, num_steps, loss.data, val_loss, elapsed, improved)
    elseif step % 10 == 0
        elapsed = time() - t_start
        @printf("step %4d / %4d | train %.4f | %.1fs\n", step, num_steps, loss.data, elapsed)
    end

    # Periodic checkpoint every 200 steps
    if step % 200 == 0
        save_checkpoint("checkpoints/checkpoint_step$(step).json", state_dict, param_keys, uchars, hyperparams;
            m_buf=m_buf, v_buf=v_buf, step=step,
            lr=lr, b1=b1, b2=b2,
            best_val_loss=best_val_loss,
            train_losses=train_loss_history, val_losses=val_loss_history,
            total_steps=step, num_steps_target=num_steps)
    end
end

elapsed = time() - t_start
@printf("\ntraining complete in %.1f seconds\n", elapsed)

# Finish W&B run
wandb_finish()

# Final save
save_checkpoint("checkpoints/final_model.json", state_dict, param_keys, uchars, hyperparams;
    m_buf=m_buf, v_buf=v_buf, step=num_steps,
    lr=lr, b1=b1, b2=b2,
    best_val_loss=best_val_loss,
    train_losses=train_loss_history, val_losses=val_loss_history,
    total_steps=num_steps, num_steps_target=num_steps)

---
## 8. Inference — Hallucinated Philosophy

Generate new philosophy-like text using temperature-controlled sampling.

In [ ]:
function generate_text(state_dict, uchars, BOS, n_layer, n_head, head_dim, block_size;
                       temperature=0.8, max_tokens=128)
    kv_keys = [Vector{Vector{Value}}() for _ in 1:n_layer]
    kv_vals = [Vector{Vector{Value}}() for _ in 1:n_layer]
    token_id = BOS
    sample = Char[]
    limit = min(max_tokens, block_size)
    for pos in 1:limit
        logits = gpt(token_id, pos, kv_keys, kv_vals, state_dict, n_layer, n_head, head_dim)
        scaled = [l / temperature for l in logits]
        probs = softmax_v(scaled)
        weights = [p.data for p in probs]
        r = rand()
        cum = 0.0
        token_id = 1
        for (idx, w) in enumerate(weights)
            cum += w
            if r <= cum
                token_id = idx
                break
            end
        end
        token_id == BOS && break
        push!(sample, uchars[token_id])
    end
    return String(sample)
end

println("--- inference (hallucinated philosophy) ---")
for i in 1:20
    text = generate_text(state_dict, uchars, BOS, n_layer, n_head, head_dim, block_size; temperature=0.8)
    @printf("sample %2d: %s\n", i, text)
end

---
## 8a. Push Model to HuggingFace Hub

Push your trained checkpoint to HuggingFace for persistence across Colab sessions.  
Set `HF_REPO_ID` in the login cell above (e.g. `"yourusername/microgpt-philosophy"`).

In [ ]:
# Push checkpoint to HuggingFace Hub
# Make sure HF_REPO_ID is set in the login cell above

if @isdefined(HF_REPO_ID) && !isempty(HF_REPO_ID)
    # Create repo if it doesn't exist yet
    hf_create_repo(HF_REPO_ID)

    # Push best model checkpoint
    if isfile("checkpoints/best_model.json")
        hf_push_checkpoint(HF_REPO_ID; checkpoint_path="checkpoints/best_model.json")
    else
        println("No best_model.json found — train first!")
    end

    # Also push final model if it exists
    if isfile("checkpoints/final_model.json")
        hf_push(HF_REPO_ID, "checkpoints/final_model.json")
    end

    println("\nDone! View your model at: https://huggingface.co/$HF_REPO_ID")
else
    println("Set HF_REPO_ID in the login cell (e.g. \"yourusername/microgpt-philosophy\")")
end

---
## 8b. Pull Checkpoint from HuggingFace Hub

Download a previously pushed checkpoint to resume training in a new Colab session.  
Run this before the Resume Training cell below.

In [ ]:
# Pull checkpoint from HuggingFace to resume training
# Make sure HF_REPO_ID is set in the login cell above

if @isdefined(HF_REPO_ID) && !isempty(HF_REPO_ID)
    mkpath("checkpoints")
    hf_pull(HF_REPO_ID, "best_model.json"; local_dir="checkpoints")
    println("\nReady to resume from checkpoints/best_model.json")
    println("Run the 'Resume Training' cell below.")
else
    println("Set HF_REPO_ID in the login cell (e.g. \"yourusername/microgpt-philosophy\")")
end

---
## 9. Resume Training from Checkpoint

Load a saved checkpoint and continue training for more steps.  
Skip this cell if you're training from scratch above.

In [ ]:
# ── Load checkpoint ──
# Change the path to load a different checkpoint
RESUME_FROM = "checkpoints/best_model.json"
EXTRA_STEPS = clamp(length(train_docs), 500, 25000)  # ~1 extra epoch

ckpt = load_checkpoint(RESUME_FROM)
state_dict = ckpt.state_dict
uchars = ckpt.uchars
BOS = ckpt.BOS
n_layer = ckpt.n_layer
n_embd = ckpt.n_embd
block_size = ckpt.block_size
n_head = ckpt.n_head
head_dim = ckpt.head_dim

hyperparams = Dict{String,Any}(
    "n_layer" => n_layer, "n_embd" => n_embd,
    "block_size" => block_size, "n_head" => n_head
)

# Reconstruct dataset and split
docs = shuffle(TRAINING_DATA)
split_idx = max(1, Int(floor(0.9 * length(docs))))
train_docs = docs[1:split_idx]
val_docs = docs[split_idx+1:end]
if isempty(val_docs)
    val_docs = docs[max(1, end-4):end]
    train_docs = docs[1:max(1, end-5)]
end

param_keys = get_param_keys(n_layer)
params = flatten_params(state_dict, param_keys)

# Restore optimizer
lr = ckpt.lr; b1 = ckpt.b1; b2 = ckpt.b2; eps = 1e-8
m_buf = length(ckpt.m_buf) == length(params) ? copy(ckpt.m_buf) : zeros(length(params))
v_buf = length(ckpt.v_buf) == length(params) ? copy(ckpt.v_buf) : zeros(length(params))

start_step = ckpt.step + 1
end_step = ckpt.step + EXTRA_STEPS
best_val_loss = ckpt.best_val_loss
train_loss_history = copy(ckpt.train_losses)
val_loss_history = copy(ckpt.val_losses)

# ── Initialize W&B logging (if API key is set) ──
if haskey(ENV, "WANDB_API_KEY") && !isempty(ENV["WANDB_API_KEY"])
    wandb_init()
end

println("\nResuming from step $(ckpt.step) -> training to step $end_step")
println("Best val loss so far: $(round(best_val_loss, digits=4))")
t_start = time()

for step in start_step:end_step
    doc = train_docs[mod1(step, length(train_docs))]
    tokens = vcat([BOS], [findfirst(==(ch), uchars) for ch in doc], [BOS])
    n = min(block_size, length(tokens) - 1)

    kv_keys = [Vector{Vector{Value}}() for _ in 1:n_layer]
    kv_vals = [Vector{Vector{Value}}() for _ in 1:n_layer]
    losses = Value[]
    for pos in 1:n
        token_id  = tokens[pos]
        target_id = tokens[pos + 1]
        logits = gpt(token_id, pos, kv_keys, kv_vals, state_dict, n_layer, n_head, head_dim)
        probs  = softmax_v(logits)
        push!(losses, -log(probs[target_id]))
    end
    loss = (1 / n) * sum(losses)
    push!(train_loss_history, loss.data)

    backward!(loss)

    lr_t = lr * (1 - (step - 1) / end_step)
    for (i, p) in enumerate(params)
        m_buf[i] = b1 * m_buf[i] + (1 - b1) * p.grad
        v_buf[i] = b2 * v_buf[i] + (1 - b2) * p.grad^2
        m_hat = m_buf[i] / (1 - b1^step)
        v_hat = v_buf[i] / (1 - b2^step)
        p.data -= lr_t * m_hat / (sqrt(v_hat) + eps)
        p.grad = 0.0
    end

    if step % 50 == 0
        val_loss = compute_val_loss(val_docs, uchars, BOS, block_size, state_dict, n_layer, n_head, head_dim)
        push!(val_loss_history, val_loss)
        elapsed = time() - t_start

        # Log to W&B
        wandb_log(; step=step, train_loss=loss.data, val_loss=val_loss, lr=lr_t)

        improved = ""
        if val_loss < best_val_loss
            best_val_loss = val_loss
            save_checkpoint("checkpoints/best_model.json", state_dict, param_keys, uchars, hyperparams;
                m_buf=m_buf, v_buf=v_buf, step=step,
                lr=lr, b1=b1, b2=b2,
                best_val_loss=best_val_loss,
                train_losses=train_loss_history, val_losses=val_loss_history,
                total_steps=step, num_steps_target=end_step)
            improved = " << new best!"
        end
        @printf("step %4d / %4d | train %.4f | val %.4f | %.1fs%s\n",
                step, end_step, loss.data, val_loss, elapsed, improved)
    elseif step % 10 == 0
        elapsed = time() - t_start
        @printf("step %4d / %4d | train %.4f | %.1fs\n", step, end_step, loss.data, elapsed)
    end
end

elapsed = time() - t_start
@printf("\nresume training complete in %.1f seconds\n", elapsed)

# Finish W&B run
wandb_finish()

save_checkpoint("checkpoints/final_model.json", state_dict, param_keys, uchars, hyperparams;
    m_buf=m_buf, v_buf=v_buf, step=end_step,
    lr=lr, b1=b1, b2=b2,
    best_val_loss=best_val_loss,
    train_losses=train_loss_history, val_losses=val_loss_history,
    total_steps=end_step, num_steps_target=end_step)

---
## 10. Download Checkpoint

Download the best model checkpoint to use with the inference server.  
In Colab, use the Files panel (left sidebar) to download, or copy to Google Drive.

In [ ]:
# List saved checkpoints
if isdir("checkpoints")
    files = readdir("checkpoints")
    println("Saved checkpoints:")
    for f in files
        path = joinpath("checkpoints", f)
        size_kb = round(filesize(path) / 1024, digits=1)
        println("  $path ($(size_kb) KB)")
    end
else
    println("No checkpoints directory found. Train first!")
end